In [1]:
%matplotlib inline
import os
import numpy as np
import agent
import env
import skimage
import skimage.io
import skimage.transform
from matplotlib import pyplot as plt
import random
import tensorflow as tf
import sys
sys.path.append('tensorflow-vgg/')
import vgg16

data_dir = '/home/vagrant/ocm/language-learning/data'

os.chdir('/home/vagrant/ocm/language-learning/code')

In [2]:
num_words = 2
vocab = ['W'+str(i) for i in range(num_words)]

vocab = ['Catword', 'Dogword']
embed_dim = 2
print(vocab)

['Catword', 'Dogword']


In [3]:
logs_path = os.path.join('..','logs')

writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())
print(logs_path)

Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
../logs


In [4]:
def load_image(path):
    # load image
    img = skimage.io.imread(path)
    img = img / 255.0
    assert (0 <= img).all() and (img <= 1.0).all()
    # print "Original Image Shape: ", img.shape
    # we crop image from center
    short_edge = min(img.shape[:2])
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
    # resize to 224, 224
    resized_img = skimage.transform.resize(crop_img, (224, 224))
    return resized_img.astype(np.float32)

In [5]:
#sender.show_images(target, distractor)

iterations = 1000

img_dirs = ['cat', 'dog']

os.getcwd()

'/home/vagrant/ocm/language-learning/code'

In [6]:
def get_image_activations(sess, vgg, image, placeholder):
    #image_pl = tf.placeholder("float32", [1, 224, 224, 3])
    batch = image.reshape((1, 224, 224, 3))
    feed_dict = {placeholder: batch}
    
    with tf.name_scope("content_vgg"):
        fc8 = sess.run(vgg.fc8, feed_dict=feed_dict)
    
    return(fc8)


In [7]:
def make_epsilon_greedy_policy(estimator, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function approximator and epsilon.

    Args:
        estimator: An estimator that returns q values for a given state
        nA: Number of actions in the environment.

    Returns:
        A function that takes the (sess, observation, epsilon) as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.

    """
    def policy_fn(sess, observation, epsilon):
        A = np.ones(nA, dtype=float) * epsilon / nA
        q_values = estimator.predict(sess, np.expand_dims(observation, 0))[0]
        best_action = np.argmax(q_values)
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [8]:
def shuffle_image_activations(im_acts):
    reordering = np.array(range(len(im_acts)))    
    random.shuffle(reordering)
    target_ind = np.argmin(reordering)
    shuffled = im_acts[reordering]
    return (shuffled, target_ind)    

In [9]:
tf.reset_default_graph()

agents = agent.Agents(vocab, image_embedding_dim = 2, embedding_dim = 2)
game = env.Environment(data_dir, img_dirs, 2)

## Run the iterations of the game
iterations = 2000
num_classes = len(img_dirs)

replay_memory_size = 50
sender_replay_memory = []
reciever_replay_memory = []
replay_memory = []
init_replay_size = 20

wins = 0
losses = 0

update_estimators_every = 50

with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.7)))) as sess:
    vgg = vgg16.Vgg16()
    
    image_pl = tf.placeholder("float32", [1, 224, 224, 3])
    vgg.build(image_pl)
    sess.run(tf.global_variables_initializer())
    
    print('Initializing replay memory...')
    for n in range(init_replay_size):
        ## Sender replay
        continue
        t,d = game.get_images()
        ta = get_image_activations(sess, vgg, t, image_pl)
        da = get_image_activations(sess, vgg, d, image_pl)        
        cw = sender.show_images(sess, ta, da)
        im_acts = np.array([ta, da])
        shuffled, target_ind = shuffle_image_activations(im_acts)
        selected = reciever.show_images(sess, cw, shuffled[0], shuffled[1])
        score = 0
        if selected==target_ind:
            score = 1
        replay_memory.append((ta, da, cw, score))
    
    print('Replay memory initialized (not yet)')
        
    
    tot_reward = 0
    for i in range(iterations):
        
        print("\rEpisode {}/{}".format(i, iterations), end="")
        sys.stdout.flush()
        
        if i % 10 == 0:
            print('last 10 interations performance ', tot_reward/10.0)
            tot_reward = 0
                    
        target_image, distractor_image = game.get_images()
        target_class = game.target_class
        #print('the target class is ', target_class)
        target_acts = get_image_activations(sess, vgg, target_image, image_pl)
        distractor_acts = get_image_activations(sess, vgg, distractor_image, image_pl)
        
        reordering = np.array([0,1])
        random.shuffle(reordering)
        target_ind = np.where(reordering==0)[0]
        
        img_array = [target_acts, distractor_acts] 
        i1, i2 = [img_array[reordering[i]] for i, img in enumerate(img_array)]

        shuffled_acts = np.concatenate([i1, i2], axis=1)
        
        ## for Sender - take action in reinforcement learning terms
        
        target_ind = 0
        reward = agents.show_images(sess, img_array, target_ind, target_class)
        tot_reward += reward
        selection = 0
        #agents.call_trial(sess, img_array, target_ind)

        if target_ind == selection:
            wins += 1
        else:
            losses += 1

    print(wins, losses)    

now building the learning graph
finished building the learning graph


/home/vagrant/anaconda2/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


build model finished: 0s
Initializing replay memory...
Replay memory initialized (not yet)
Episode 0/2000last 10 interations performance  0.0
dog Catword 0
Episode 1/2000cat Dogword 0
Episode 2/2000cat Catword 1.0
Episode 3/2000dog Dogword 1.0
Episode 4/2000cat Dogword 0
Episode 5/2000cat Dogword 0
Episode 6/2000dog Catword 0
Episode 7/2000dog Catword 0
Episode 8/2000dog Dogword 1.0
Episode 9/2000dog Dogword 1.0
Episode 10/2000last 10 interations performance  0.4
dog Catword 0
Episode 11/2000cat Dogword 0
Episode 12/2000cat Dogword 0
Episode 13/2000dog Dogword 1.0
Episode 14/2000cat Dogword 0
Episode 15/2000dog Catword 0
Episode 16/2000dog Dogword 1.0
Episode 17/2000dog Dogword 1.0
Episode 18/2000dog Dogword 1.0
Episode 19/2000dog Dogword 1.0
Episode 20/2000last 10 interations performance  0.5
dog Dogword 1.0
Episode 21/2000dog Dogword 1.0
Episode 22/2000cat Dogword 0
Episode 23/2000dog Dogword 1.0
Episode 24/2000cat Dogword 0
Episode 25/2000dog Dogword 1.0
Episode 26/2000cat Catword 1

In [10]:
tf.__version__

'0.12.1'